# Tensors
https://pytorch.org/tutorials/beginner/pytorch_with_examples.html  

## A one-hidden-layer MLP in numpy

Warm-up: numpy

Before introducing PyTorch, we will first implement the network using numpy.

Numpy provides an n-dimensional array object, and many functions for manipulating these arrays. Numpy is a generic framework for scientific computing; it does not know anything about computation graphs, or deep learning, or gradients. However we can easily use numpy to fit a two-layer network to random data by manually implementing the forward and backward passes through the network using numpy operations:

In [1]:
import numpy
import os
import torch

In [2]:
# -*- coding: utf-8 -*-
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy() # grad_h is simply 1 * grad_h_relu
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

(0, 24034803.324259833)
(1, 17004251.18245837)
(2, 13107106.282321503)
(3, 10494348.218092594)
(4, 8439672.934316734)
(5, 6729815.718802508)
(6, 5273912.754218644)
(7, 4082793.780899251)
(8, 3127898.643416139)
(9, 2393623.413513685)
(10, 1836710.2479884159)
(11, 1422879.1556859976)
(12, 1115529.1742084555)
(13, 887724.592885829)
(14, 717318.2875885027)
(15, 588630.0108371691)
(16, 490030.41710034094)
(17, 413360.81157165486)
(18, 352738.8408941267)
(19, 303984.3090071075)
(20, 264192.4101825962)
(21, 231323.53491322376)
(22, 203779.63924326916)
(23, 180418.07499144727)
(24, 160422.84266630426)
(25, 143136.5922679684)
(26, 128148.88347777566)
(27, 115094.47936238786)
(28, 103668.8644046945)
(29, 93586.94774201732)
(30, 84659.83991334098)
(31, 76745.54034958303)
(32, 69699.95551567645)
(33, 63403.92935180539)
(34, 57765.070140963944)
(35, 52699.383708381916)
(36, 48139.89055847437)
(37, 44027.91897762392)
(38, 40316.05030457552)
(39, 36957.22713863536)
(40, 33912.78150078796)
(41, 31149.

(462, 1.869571398375307e-05)
(463, 1.7894317966223235e-05)
(464, 1.7127486750818837e-05)
(465, 1.63938496682843e-05)
(466, 1.5691522482630934e-05)
(467, 1.5019346556348542e-05)
(468, 1.4376237071557792e-05)
(469, 1.3761111005412206e-05)
(470, 1.3172173116963056e-05)
(471, 1.2608419725185125e-05)
(472, 1.206901246130377e-05)
(473, 1.155284042152218e-05)
(474, 1.105873314979355e-05)
(475, 1.0585797824374265e-05)
(476, 1.0133277146348525e-05)
(477, 9.700227842747192e-06)
(478, 9.285736059726316e-06)
(479, 8.888937374888538e-06)
(480, 8.509301895121985e-06)
(481, 8.145867095107694e-06)
(482, 7.798118764657362e-06)
(483, 7.465166700605445e-06)
(484, 7.1465697726207264e-06)
(485, 6.841549765976264e-06)
(486, 6.549746677279296e-06)
(487, 6.270376255117114e-06)
(488, 6.002992765720278e-06)
(489, 5.747007252197061e-06)
(490, 5.5020272068166344e-06)
(491, 5.267526454894014e-06)
(492, 5.043062912882867e-06)
(493, 4.828149815652275e-06)
(494, 4.622473879920585e-06)
(495, 4.425633355048186e-06)
(49

## The forward pass involves the following functions:

1. $$x \dot{} w1 = h$$
2. $$ h\_relu = f(h) = \left\{
        \begin{array}{ll}
            h & \quad h \gt 0 \\
            0 & \quad h \leq 0
        \end{array}
    \right. $$
3. $$y\_pred = h\_relu \dot{} w2 $$
4. $$error = \sum_{n=1}^{i} (y\_pred_i - y_i)^2 $$

## We compute two gradients for the backward pass: w2 and w1

First, the gradient for w2.  The following equation describes this gradient.  
$$ \frac{\partial error}{\partial w2} = \frac{\partial error}{\partial y\_pred} * \frac{\partial y\_pred}{\partial w2} $$

We can simplify the computation by braking it down into two steps:

1. $$ \frac{\partial error}{\partial w2} = 2(y\_pred - y)* \frac{\partial y\_pred}{\partial w2} $$
2. $$ \frac{\partial y\_pred}{\partial w2} = h\_relu $$  

The solution is just the product of parts 1 and 2:  

$$ \frac{\partial error}{\partial w2} = 2(y\_pred - y)*h\_relu $$ 

Following the same logic, we can compute the gradient for w1 as the following:  
$$ \frac{\partial error}{\partial w1} = \frac{\partial error}{\partial y\_pred} * \frac{\partial y\_pred}{\partial h\_relu} * \frac{\partial h\_relu}{\partial h}* \frac{\partial h}{\partial w1}$$

1. $$ \frac{\partial error}{\partial w2} = 2(y\_pred - y)* \frac{\partial y\_pred}{\partial h\_relu} * \frac{\partial h\_relu}{\partial h}* \frac{\partial h}{\partial w1} $$
2. $$ \frac{\partial y\_pred}{\partial h\_relu} = w2* \frac{\partial h\_relu}{\partial h}* \frac{\partial h}{\partial w1} $$  
3. $$\frac{\partial h\_relu}{\partial h} = \left\{
        \begin{array}{ll}
            1* \frac{\partial h}{\partial w1} & \quad h \gt 0 \\
            0 & \quad h \leq 0
        \end{array}
    \right. $$
4. $$\frac{\partial h}{\partial w1} = x$$

The solution is the product of parts 1, 2, 3, and 4:  

$$ \frac{\partial error}{\partial w2} = 2(y\_pred - y) * w2 * 1 * x$$ 

## PyTorch: Tensors

Numpy is a great framework, but it cannot utilize GPUs to accelerate its numerical computations. For modern deep neural networks, GPUs often provide speedups of 50x or greater, so unfortunately numpy won’t be enough for modern deep learning.

Here we introduce the most fundamental PyTorch concept: the Tensor. A PyTorch Tensor is conceptually identical to a numpy array: a Tensor is an n-dimensional array, and PyTorch provides many functions for operating on these Tensors. Like numpy arrays, PyTorch Tensors do not know anything about deep learning or computational graphs or gradients; they are a generic tool for scientific computing.

However unlike numpy, PyTorch Tensors can utilize GPUs to accelerate their numeric computations. To run a PyTorch Tensor on GPU, you simply need to cast it to a new datatype.

Here we use PyTorch Tensors to fit a two-layer network to random data. Like the numpy example above we need to manually implement the forward and backward passes through the network:

In [3]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
# device = torch.device("cpu") # Uncomment this to run on CPU
device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(50000):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

(0, 23955730.0)
(1, 18764744.0)
(2, 17730712.0)
(3, 18142756.0)
(4, 18421954.0)
(5, 17352852.0)
(6, 14760932.0)
(7, 11149851.0)
(8, 7670963.5)
(9, 4923366.0)
(10, 3085702.5)
(11, 1944661.75)
(12, 1271368.375)
(13, 872854.1875)
(14, 633439.1875)
(15, 483633.34375)
(16, 385396.125)
(17, 317179.9375)
(18, 267276.125)
(19, 229043.109375)
(20, 198647.671875)
(21, 173762.15625)
(22, 152965.40625)
(23, 135323.96875)
(24, 120200.1953125)
(25, 107111.1640625)
(26, 95715.84375)
(27, 85750.21875)
(28, 76992.03125)
(29, 69271.2421875)
(30, 62438.4609375)
(31, 56379.23828125)
(32, 50999.13671875)
(33, 46205.1484375)
(34, 41922.20703125)
(35, 38089.3515625)
(36, 34655.40234375)
(37, 31572.54296875)
(38, 28801.6640625)
(39, 26307.29296875)
(40, 24057.54296875)
(41, 22024.025390625)
(42, 20183.865234375)
(43, 18517.1484375)
(44, 17006.4453125)
(45, 15633.2822265625)
(46, 14384.0234375)
(47, 13246.615234375)
(48, 12209.8408203125)
(49, 11264.634765625)
(50, 10402.2001953125)
(51, 9613.484375)
(52, 8891

(406, 0.001662678550928831)
(407, 0.0016127779381349683)
(408, 0.0015658298507332802)
(409, 0.0015187464887276292)
(410, 0.0014727002708241343)
(411, 0.001428149058483541)
(412, 0.0013892808929085732)
(413, 0.0013470579870045185)
(414, 0.0013072011061012745)
(415, 0.0012709024595096707)
(416, 0.0012338150991126895)
(417, 0.0011985335731878877)
(418, 0.001165737514384091)
(419, 0.0011319450568407774)
(420, 0.001100372988730669)
(421, 0.0010683807777240872)
(422, 0.0010401466861367226)
(423, 0.0010112125892192125)
(424, 0.0009838127298280597)
(425, 0.0009570303373038769)
(426, 0.0009306095307692885)
(427, 0.000906734203454107)
(428, 0.0008807011181488633)
(429, 0.0008576830732636154)
(430, 0.0008343859808519483)
(431, 0.0008128652116283774)
(432, 0.0007914408924989402)
(433, 0.0007717997068539262)
(434, 0.0007501500658690929)
(435, 0.0007311097579076886)
(436, 0.0007114177569746971)
(437, 0.0006944510387256742)
(438, 0.0006772582419216633)
(439, 0.000659596174955368)
(440, 0.000643522595

(913, 9.454351129534189e-06)
(914, 9.470581971982028e-06)
(915, 9.428945304534864e-06)
(916, 9.37138611334376e-06)
(917, 9.395233064424247e-06)
(918, 9.394771950610448e-06)
(919, 9.332887202617712e-06)
(920, 9.288559340348002e-06)
(921, 9.219851563102566e-06)
(922, 9.241588486474939e-06)
(923, 9.189735465042759e-06)
(924, 9.15783311938867e-06)
(925, 9.103705451707356e-06)
(926, 9.131517799687572e-06)
(927, 9.064834557648282e-06)
(928, 9.054894690052606e-06)
(929, 8.971203897090163e-06)
(930, 8.961576895671897e-06)
(931, 8.891043762560003e-06)
(932, 8.874306331563275e-06)
(933, 8.847516255627852e-06)
(934, 8.796691872703377e-06)
(935, 8.822521522233728e-06)
(936, 8.784816600382328e-06)
(937, 8.766566679696552e-06)
(938, 8.728022294235416e-06)
(939, 8.712292583368253e-06)
(940, 8.68608276505256e-06)
(941, 8.651637472212315e-06)
(942, 8.642189641250297e-06)
(943, 8.61948592500994e-06)
(944, 8.584559509472456e-06)
(945, 8.547523066226859e-06)
(946, 8.51714958116645e-06)
(947, 8.49678326630

(1444, 3.154546675432357e-06)
(1445, 3.156323145958595e-06)
(1446, 3.1596312055626186e-06)
(1447, 3.1470510748476954e-06)
(1448, 3.1353799840871943e-06)
(1449, 3.1457539080292918e-06)
(1450, 3.1349193250207463e-06)
(1451, 3.1360880257125245e-06)
(1452, 3.1356048566522077e-06)
(1453, 3.126600404357305e-06)
(1454, 3.1261283766070846e-06)
(1455, 3.1096874408831354e-06)
(1456, 3.1053784823598107e-06)
(1457, 3.0993737709650304e-06)
(1458, 3.081848717556568e-06)
(1459, 3.0812780096312054e-06)
(1460, 3.0828180115349824e-06)
(1461, 3.0777205211052205e-06)
(1462, 3.0767987482249737e-06)
(1463, 3.066062618017895e-06)
(1464, 3.061941242776811e-06)
(1465, 3.052490455957013e-06)
(1466, 3.0474393497570418e-06)
(1467, 3.051601879633381e-06)
(1468, 3.0544003948307363e-06)
(1469, 3.040289811906405e-06)
(1470, 3.0371434149856213e-06)
(1471, 3.0307746783364564e-06)
(1472, 3.0341041110659717e-06)
(1473, 3.046954589080997e-06)
(1474, 3.0332628284668317e-06)
(1475, 3.019571295226342e-06)
(1476, 3.0112860258

(1969, 1.7198200339407776e-06)
(1970, 1.7177399058709852e-06)
(1971, 1.7164064729513484e-06)
(1972, 1.7186214336106787e-06)
(1973, 1.7153686258097878e-06)
(1974, 1.719032979963231e-06)
(1975, 1.7144634512078483e-06)
(1976, 1.7116499293479137e-06)
(1977, 1.7124204987339908e-06)
(1978, 1.7188931451528333e-06)
(1979, 1.7085267245420255e-06)
(1980, 1.7067636690626387e-06)
(1981, 1.6990175026876386e-06)
(1982, 1.6991632492135977e-06)
(1983, 1.700391180747829e-06)
(1984, 1.7006180996759213e-06)
(1985, 1.6921609358178102e-06)
(1986, 1.6856067759363214e-06)
(1987, 1.6825892998895142e-06)
(1988, 1.6811447949294234e-06)
(1989, 1.6844160199980251e-06)
(1990, 1.6842280956552713e-06)
(1991, 1.6803770677142893e-06)
(1992, 1.678268290561391e-06)
(1993, 1.6817331243146327e-06)
(1994, 1.671606355557742e-06)
(1995, 1.6822340285216342e-06)
(1996, 1.6838932879181812e-06)
(1997, 1.6870774288690882e-06)
(1998, 1.6985615047815372e-06)
(1999, 1.6982166926027276e-06)
(2000, 1.6934393443079898e-06)
(2001, 1.683

...

(47651, 4.375556983404749e-08)
(47652, 4.335146996936601e-08)
(47653, 4.420906662971902e-08)
(47654, 4.3963275686564884e-08)
(47655, 4.332049030608687e-08)
(47656, 4.352578741873003e-08)
(47657, 4.321302071730315e-08)
(47658, 4.339197801073169e-08)
(47659, 4.513152163099221e-08)
(47660, 4.475021597727391e-08)
(47661, 4.436461154000426e-08)
(47662, 4.4785537056668545e-08)
(47663, 4.432018840816454e-08)
(47664, 4.4677847199636744e-08)
(47665, 4.4498619899968617e-08)
(47666, 4.504346051703578e-08)
(47667, 4.4822765943308696e-08)
(47668, 4.4553328137908466e-08)
(47669, 4.534714648229965e-08)
(47670, 4.501286454683395e-08)
(47671, 4.546529552840184e-08)
(47672, 4.558989630254473e-08)
(47673, 4.557926658321776e-08)
(47674, 4.570170730744394e-08)
(47675, 4.577986700837755e-08)
(47676, 4.604069303582037e-08)
(47677, 4.5378197199852366e-08)
(47678, 4.6190034908022426e-08)
(47679, 4.558340194193988e-08)
(47680, 4.5693905548205294e-08)
(47681, 4.553764298975693e-08)
(47682, 4.496574135259834e-08)

(48179, 4.8437826194458466e-08)
(48180, 4.867499114880047e-08)
(48181, 4.943549924973922e-08)
(48182, 4.926819485717715e-08)
(48183, 4.9465114670965704e-08)
(48184, 4.941139764014224e-08)
(48185, 4.898130612218665e-08)
(48186, 4.9475474384053086e-08)
(48187, 5.041164286012645e-08)
(48188, 4.9522839162818855e-08)
(48189, 4.88609046556121e-08)
(48190, 4.894162231039445e-08)
(48191, 4.90633809135943e-08)
(48192, 4.9317911532398284e-08)
(48193, 4.923658281086318e-08)
(48194, 4.9304652804949e-08)
(48195, 4.955374066639706e-08)
(48196, 4.9834774529244896e-08)
(48197, 4.97745773486713e-08)
(48198, 4.988144297612962e-08)
(48199, 5.0098499571049615e-08)
(48200, 4.9893067455286655e-08)
(48201, 4.872698156077604e-08)
(48202, 4.9582929761982086e-08)
(48203, 4.8832440313617553e-08)
(48204, 4.825127319918465e-08)
(48205, 4.8653660655872955e-08)
(48206, 4.914001294764603e-08)
(48207, 4.846880585773761e-08)
(48208, 4.87059566012249e-08)
(48209, 4.8773259209156095e-08)
(48210, 4.828974908832606e-08)
(4

(48717, 4.978930689958361e-08)
(48718, 4.944185150179692e-08)
(48719, 4.9998007511931064e-08)
(48720, 4.9848992489387456e-08)
(48721, 5.019761317726079e-08)
(48722, 5.029225746966404e-08)
(48723, 5.0376613103253476e-08)
(48724, 5.066434027867217e-08)
(48725, 5.0874561452474154e-08)
(48726, 4.990504010038421e-08)
(48727, 4.958387478382065e-08)
(48728, 4.9600586748965725e-08)
(48729, 4.916476825655991e-08)
(48730, 4.873901104929246e-08)
(48731, 4.867784753059823e-08)
(48732, 4.871968428687978e-08)
(48733, 4.943346709751495e-08)
(48734, 5.058354446418889e-08)
(48735, 4.990415192196451e-08)
(48736, 5.064850228109208e-08)
(48737, 4.9816385683243425e-08)
(48738, 5.0247543015302654e-08)
(48739, 4.9682896019476175e-08)
(48740, 4.986645762983244e-08)
(48741, 4.9845709781948244e-08)
(48742, 4.987265356248827e-08)
(48743, 5.0241226290381746e-08)
(48744, 5.044404360887711e-08)
(48745, 5.038367589804693e-08)
(48746, 5.016738668928156e-08)
(48747, 5.098227262578803e-08)
(48748, 5.067031594307991e-08

(49224, 4.442180312480559e-08)
(49225, 4.3481442446591245e-08)
(49226, 4.3909306413070226e-08)
(49227, 4.338559023153721e-08)
(49228, 4.391116803503792e-08)
(49229, 4.371994322127648e-08)
(49230, 4.305078249444705e-08)
(49231, 4.364832051351186e-08)
(49232, 4.3832038443269994e-08)
(49233, 4.359905148021426e-08)
(49234, 4.4165211932067905e-08)
(49235, 4.4036134738689725e-08)
(49236, 4.4473601690242504e-08)
(49237, 4.416720145172803e-08)
(49238, 4.485841742507546e-08)
(49239, 4.438725298427926e-08)
(49240, 4.4149992106667924e-08)
(49241, 4.504489581336202e-08)
(49242, 4.377411144673715e-08)
(49243, 4.486327398467438e-08)
(49244, 4.471275261153096e-08)
(49245, 4.4591335068844273e-08)
(49246, 4.5235143630861785e-08)
(49247, 4.564964228848112e-08)
(49248, 4.5810509163857205e-08)
(49249, 4.552821053493972e-08)
(49250, 4.4501973661681404e-08)
(49251, 4.4445698677009204e-08)
(49252, 4.438410883267352e-08)
(49253, 4.5336975063037244e-08)
(49254, 4.511568363341212e-08)
(49255, 4.601702130457852e

(49753, 4.90436562472496e-08)
(49754, 4.921600549323557e-08)
(49755, 5.035710870515686e-08)
(49756, 4.9186347439444944e-08)
(49757, 4.9019746484191273e-08)
(49758, 4.91424145820929e-08)
(49759, 4.941844622408098e-08)
(49760, 4.915878548672481e-08)
(49761, 4.910360473786568e-08)
(49762, 4.8753946657598135e-08)
(49763, 4.902161521158632e-08)
(49764, 4.922406304785909e-08)
(49765, 4.847205303804003e-08)
(49766, 4.861374947040531e-08)
(49767, 4.863220937068036e-08)
(49768, 4.7881336229238514e-08)
(49769, 4.874444670122102e-08)
(49770, 4.940250875051788e-08)
(49771, 4.9736648577436426e-08)
(49772, 4.905179906700141e-08)
(49773, 4.9019384107396036e-08)
(49774, 4.8656325191132055e-08)
(49775, 4.7879261444450094e-08)
(49776, 4.7651376178237115e-08)
(49777, 4.777093209895611e-08)
(49778, 4.7784716628029855e-08)
(49779, 4.7856836715709505e-08)
(49780, 4.857974289507183e-08)
(49781, 4.85960711671396e-08)
(49782, 4.8851660494619864e-08)
(49783, 4.884154236606264e-08)
(49784, 4.9428919624006085e-08

## Doing the same thing, but using pytorch's Autograd Capabilities


In [4]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("cpu")
# dtype = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

(0, 31548472.0)
(1, 28293014.0)
(2, 28417600.0)
(3, 27130712.0)
(4, 22559472.0)
(5, 15694898.0)
(6, 9458009.0)
(7, 5280842.0)
(8, 2998015.75)
(9, 1838158.625)
(10, 1249345.0)
(11, 928834.5625)
(12, 735993.0)
(13, 606895.3125)
(14, 512462.5625)
(15, 439016.78125)
(16, 379593.21875)
(17, 330393.78125)
(18, 289086.21875)
(19, 254251.203125)
(20, 224460.671875)
(21, 198773.4375)
(22, 176536.96875)
(23, 157217.359375)
(24, 140362.171875)
(25, 125597.7109375)
(26, 112639.6953125)
(27, 101238.875)
(28, 91174.7265625)
(29, 82273.59375)
(30, 74362.25)
(31, 67319.90625)
(32, 61043.0234375)
(33, 55431.75390625)
(34, 50406.25390625)
(35, 45894.765625)
(36, 41838.03125)
(37, 38187.296875)
(38, 34893.95703125)
(39, 31920.31640625)
(40, 29228.46875)
(41, 26791.2265625)
(42, 24581.423828125)
(43, 22574.265625)
(44, 20749.48828125)
(45, 19087.0703125)
(46, 17571.763671875)
(47, 16189.5234375)
(48, 14926.9228515625)
(49, 13773.14453125)
(50, 12718.123046875)
(51, 11752.2255859375)
(52, 10866.5048828125)

In [5]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode